In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras.utils import np_utils
from keras.layers import Dense, Activation,Flatten, Convolution2D, Dropout, MaxPooling2D
from keras.models import Sequential

import datetime